In [ ]:
##-----degraded-----##
# !pip install -Iv gym==0.17.1
# !apt-get install python-opengl -y
# !apt install xvfb -y
# !pip install gym[atari]
# !pip install pyvirtualdisplay
# !conda install piglet
# !pip install pystan
# !conda install swig
# #!pip install box2d-py
# !pip install box2d-py
# !pip install gym[box2d]

In [ ]:
# Upload 'HC ROMS.zip' and 'ROMS.zip' online.
!python -m atari_py.import_roms .
import gym
env = gym.make('PongNoFrameskip-v4')
print(env.observation_space.shape)

In [ ]:
# -*- coding: utf-8 -*-

import math, random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F
import itertools
from collections import deque
from common.old_atari_wrappers import wrap_pytorch, make_atari, wrap_deepmind
import matplotlib.pyplot as plt

In [ ]:

def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in',nonlinearity='relu')
        torch.nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in', a=0.2, nonlinearity='leaky_relu')
        torch.nn.init.zeros_(m.bias)



In [ ]:

class QNetwork(nn.Module):    
    def __init__(self, input_shape, num_actions):
        super(QNetwork, self).__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions

        self.features = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU()
        )
        self.features.apply(init_weights)

        self.fc1 = nn.Linear(64*7*7, 256)
        self.fc1.apply(init_weights)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.fc2 = nn.Linear(256, num_actions)
        
        
        
    def forward(self, state):
        state = torch.div(state, 255)
        state = state.cuda()
        
        feats = self.features(state)
        fc1 = self.fc1(feats.view(feats.size(0),-1))
        leaky_relu = self.leaky_relu(fc1)

        return self.fc2(leaky_relu).cuda()

    def feature_size(self):
        with torch.no_grad():
            return self.features(torch.zeros(1, *self.input_shape)).view(1, -1).size(1)
        
        

In [ ]:

class LearningAgent: 
    def __init__(self, **kwargs):      
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using {} for training the algorithm.".format(self.device))        
        self.env = kwargs.get('training_env',None)
        self.gamma = kwargs.get('gamma', 0.99)
        self.hard_update_interval = kwargs.get('target network update interval', 10000)
        
        self.update_step = 0
        
        # ---- initialize networks ----
        self.q_net1 = QNetwork(env.observation_space.shape, env.action_space.n).to(self.device)
        self.target_q_net1 = QNetwork(env.observation_space.shape, env.action_space.n).to(self.device)


        # initialize optimizers using Adam optimizer.
        self.q1_optimizer = optim.Adam(self.q_net1.parameters(), kwargs.get('learning rate', 1e-4))
        # initialize the loss function (Huberloss) 
        self.criterion = nn.SmoothL1Loss(reduction='mean')
        
        self.replay_buffer = ReplayBuffer(kwargs.get('memory size',1000000), env.observation_space.shape)
        


    def act(self, state):
        # implement epsilon-greedy, 
        state = torch.tensor(state)
        if state.size(0) == 4:
          state = state.unsqueeze(0).to(self.device)

        with torch.no_grad():
          # Epsilon-Greedy action selection
          if np.random.random() > self.epsilon: # Exploit
            action = self.q_net1.forward(state)
            act = torch.argmax(action).item()
          else:  # Explore
            act = env.action_space.sample()
        return act


    def test_act(self, state):
        state = torch.tensor(state)
        if state.size(0) == 4:
          state = state.unsqueeze(0).to(self.device)

        with torch.no_grad():
          action = self.q_net1.forward(state)
          act = torch.argmax(action).item()
        return act
    
   
    def update(self, batch_size):       
       # Very similar to previous DDQN, except for the loss definition and update manner of target network
        self.q_net1.train()
        
        transitions = self.replay_buffer.sample(batch_size)        
        states, actions, rewards, next_states, dones = transitions
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        # Implement DDQN here, should be similar/same as DDQN in previous task.
        
        with torch.no_grad():
          #DDQN
          act_idx = self.q_net1(next_states).argmax(1).unsqueeze(1)
          next_state_act_vals = self.target_q_net1(next_states).gather(1, act_idx)
          y_i = (1 - dones)*(next_state_act_vals.squeeze(1) * self.gamma) + rewards#DDQN

        state_action_values = self.q_net1(states).gather(1, actions.unsqueeze(1)) #self.q_net1(states).gather(1, act_idx)


        # q loss using self.criterion
        q1_loss = self.criterion(state_action_values, y_i.detach().unsqueeze(1))       

        # update q networks        
        self.q1_optimizer.zero_grad()
        q1_loss.backward()
        self.q1_optimizer.step()

        # --------Perform Hard update of target networks every self.hard_update_interval-----------     
        if self.update_step % self.hard_update_interval == 0:    
          for target_param, param in zip(self.target_q_net1.parameters(), self.q_net1.parameters()):
            target_param.data.copy_(param.data)
        
        self.update_step += 1
        
        # just for monitoring Q loss, in case of wrong implementation, this could diverge to insane values.
        if self.update_step % 1000 == 999:
            print("Q loss: {}".format(q1_loss))


In [ ]:

class ReplayBuffer(object):
    
    def __init__(self, max_size, state_dim):
        self.buffer = deque(maxlen=max_size)
        self.state_dim = state_dim
    
    def push(self, state, action, reward, next_state, done):
        experience = (state, action, reward, next_state, done)
        self.buffer.append(experience)

    
    def sample(self, batch_size):

        state_batch = np.empty([batch_size,self.state_dim[0], self.state_dim[1], self.state_dim[2]])
        action_batch = np.empty([batch_size])
        reward_batch = np.empty([batch_size])
        next_state_batch = np.empty([batch_size,self.state_dim[0], self.state_dim[1], self.state_dim[2]])
        done_batch = np.empty([batch_size])
        # Take #batch_size samples from the replay buffer and write into different arrays.        
        batch = random.sample(range(len(self)),batch_size)
        for n in range(batch_size):
          (state, action, reward, next_state, done) = self.buffer[batch[n]]
          state_batch[n,:,:] = state
          action_batch[n] = action
          reward_batch[n] = reward
          next_state_batch[n,:,:] = next_state
          done_batch[n] = done
          
        
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch


    def __len__(self):
        return len(self.buffer)


def learn(agent, **kwargs):   
    #save the statistics here
    update_step = 0
    train_start = kwargs.get('number of pre-interactions to start training', 50000)
    batch_size = kwargs.get('batch size', 32)
    try:
        max_steps = env._max_episode_steps# 
    except:
        max_steps = kwargs.get('max steps per episode', 27500)   
    max_episodes = kwargs.get('max episodes', 1000000)   
    update_every_n_step = kwargs.get('update every n steps', 4)   
    n_frame = 0
    running_100_mean = np.zeros(100)    
    epsilon_start = 1
    epsilon_final = 0.1

    
    epi_returns = [] # save this statistics for plotting
    test_phase_avg_returns = []

    for episode in range(max_episodes):    
                
        state = env.reset()
        epi_return = 0
        for step in itertools.count():
            # Implement linearly-decaying epsilon
            agent.epsilon = max(epsilon_final, epsilon_start - (epsilon_start - epsilon_final)* n_frame/500000. )
            action= agent.act(state)                            
            next_state, reward, done, _ = env.step(action)            
            n_frame+=1        
            # Push the experience into replay buffer                                                              
            agent.replay_buffer.push(state, action, reward, next_state, done)
            epi_return += reward
            
            # gent collects enough experience () and performs an update every 4 interactions.
            if (len(agent.replay_buffer) > train_start) and ((n_frame % update_every_n_step) == 0): 
                agent.update(batch_size)
                update_step += 1
            state = next_state
            
            if done == True or step >= max_steps:
                running_100_mean[episode%100] = epi_return
                epi_returns.append(epi_return)
                print('Episode : {} , episodic length : {}, return: {}, exp_count :{}, averged returnover past 100 episodes : {}'.format(episode , step, epi_return, n_frame, np.mean(running_100_mean[:min(100,episode+1)]))) 
                break

        #---------------Save the satistics of ----------------
        if episode%50 == 49: # you can change here
            epi_returns_np = np.array(epi_returns)
            # ---save the statistics for plotting ---
            np.save("epi_returns.npy", epi_returns_np)
        #----------------------------Testing-----------------------
        if episode%100 == 99:
            test_returns = np.zeros(15)
            print ('-----start Testing------')
            epi_return = 0       
            state = test_env.reset()  
            for i in range(15): # test for 15 episodes
                state = test_env.reset()  
                for t in range(max_steps):
                    # env.render()
                    action = agent.test_act(state)
                    next_state, reward, done, _ = test_env.step(action)
                    state = next_state
                    epi_return += reward                  
                    if done or (t == max_steps - 1):
                        test_returns[i] = epi_return
                        epi_return = 0
                        break           
            test_phase_avg_returns.append(np.mean(test_returns))        
            print('Testing result, Returns: {}'.format(test_returns))
            test_phase_avg_returns_np = np.array(test_phase_avg_returns)
            # ---IMPORTNT: To do: save the statistics here for plotting ---
            np.save("test_phase_avg_returns.npy", test_phase_avg_returns_np)


In [ ]:

if __name__ == '__main__':  
    
    env_id = "PongNoFrameskip-v4" # could change to other games, but only use 'NoFrameskip-v4' version. Some games e.g., spaceinvaders require changing the code in wrapper, i.e., frame_skip =3
    env = make_atari(env_id)
    env = wrap_deepmind(env)
    env = wrap_pytorch(env) # Note the env returns the unnormalized pixel [0,255].
    test_env = make_atari(env_id)
    test_env = wrap_deepmind(test_env, episode_life=False, clip_rewards=False)
    test_env = wrap_pytorch(test_env)
    
    # Hint: you could check the shape and content of the state, it is not yet normlized between [0,1]
    obs = env.reset()
    print(obs.shape)
    for i in range(obs.shape[0]):
        plt.imshow(obs[i])
    
    random_seed = None
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        torch.manual_seed(random_seed)
        env.seed(random_seed)
        np.random.seed(random_seed)  
        random.seed(random_seed)
        
    # DQN params
    config = {}
    config.update({'gamma': 0.99})
    config.update({'learning rate': 1e-4})
    config.update({'memory size': 300000}) # 1M frame memory buffer takes around 8GB in RAM. For local computer, you could use 1M memory size. For Colab, size should be <= 0.3M.
    config.update({'target network update interval': 2500}) # every 10000 training steps, update the target model.
    config.update({'max episodes': 1000000}) # you don't need to run that long, but just keep running.
    config.update({'max steps per episode': 27500}) 
    config.update({'batch size': 32}) 
    config.update({'update every n steps': 4})
    config.update({'number of pre-interactions to start training': 10000})
    config.update({'training_env': env})
    config.update({'testing_env': test_env})
    #  agent
    agent = LearningAgent(**config)
    
    # train
    learn(agent, **config)

(4, 84, 84)
Using cuda for training the algorithm.
Episode : 0 , episodic length : 964, return: -19.0, exp_count :965, averged returnover past 100 episodes : -19.0
Episode : 1 , episodic length : 938, return: -21.0, exp_count :1904, averged returnover past 100 episodes : -20.0
Episode : 2 , episodic length : 905, return: -21.0, exp_count :2810, averged returnover past 100 episodes : -20.333333333333332
Episode : 3 , episodic length : 867, return: -21.0, exp_count :3678, averged returnover past 100 episodes : -20.5
Episode : 4 , episodic length : 926, return: -20.0, exp_count :4605, averged returnover past 100 episodes : -20.4
Episode : 5 , episodic length : 847, return: -21.0, exp_count :5453, averged returnover past 100 episodes : -20.5
Episode : 6 , episodic length : 1047, return: -20.0, exp_count :6501, averged returnover past 100 episodes : -20.428571428571427
Episode : 7 , episodic length : 936, return: -21.0, exp_count :7438, averged returnover past 100 episodes : -20.5
Episode :